## Load Packages

In [1]:
# # Install tempo-embeddings from GitHub
# # This can also refer to a specific version or branch

# %pip install --upgrade pip  # Required for properly resolving dependencies
# %pip uninstall -y tempo_embeddings  # Remove existing installation
# %pip install --upgrade git+https://github.com/Semantics-of-Sustainability/tempo-embeddings.git

In [2]:
# make sure installation has succeeded
import tempo_embeddings

from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

In [3]:
%load_ext autoreload

In [4]:
try:
    import google.colab

    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False

In [5]:
%autoreload now

from pathlib import Path
from tqdm import tqdm
from tempo_embeddings.text.corpus import Corpus
import ipywidgets as widgets

## Load Database Manager

The `db_path` parameter should point to the directory where the database is, so the original configuration and records are loaded. The database was created using the notebook `1_compute_embeddings_nl.ipynb`. If the given path does not exist, a new EMPTY database will be created there. 

A bigger `batch_size` could make the search faster but if it is too big you might run out of memory.

In [6]:
from tempo_embeddings.embeddings.vector_database import ChromaDatabaseManager

# Here we load only the ANP collection because metadata field names diverge across datasets
db = ChromaDatabaseManager(db_path="testing_db", batch_size=8)
db.connect()


/Users/jose/Repos/tempo-embeddings/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
03:07:45 INFO:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
03:07:45 INFO:Path 'testing_db' already exists. Loading DB configuration:
{'embedder_name': 'NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers', 'embedder_type': 'custom_model', 'existing_collections': ['anp_sg_corpus', 'anp_duurzam']}


## Choose from the available Collections in the Database

In [7]:
existing_colls = db.get_available_collections()
collection_selector = widgets.Dropdown(
    options=existing_colls,
    description='Choose a Collection:',
    disabled=False,
    style={'description_width': 'initial'} 
)
collection_selector

Dropdown(description='Choose a Collection:', options=('anp_sg_corpus', 'anp_duurzam'), style=DescriptionStyle(…

## Show number of records in the selected collection

In [8]:
collection = db.get_existing_collection(collection_selector.value)
print(f"\nCollection '{collection_selector.value}' has {collection.count()} records")

03:07:45 INFO:Retrieved existing collection 'anp_sg_corpus'



Collection 'anp_sg_corpus' has 1558 records


## Create Sub-Corpus

To make the processing and visualization easier, we will create a new `Corpus` comprising only a subet of the original Collection. This corpus will contain only the records of interest. This is done by querying the database with keyword and metadata constraints. In this example we allow to look for:

- **Filter Terms:** retrieve only passages that contain exactly the given keywords.
- **Year Range:** retrieve only the records which are inside the provided years
- **Neighbors:** This indicates how much to *expand* the search into more datapoints. The idea is to retrieve the *top_k* neighbors of the initially retrieved passages. Ideally this will give related passages that did not mention any of the keywords explicitly.

In [9]:
widget_year_range=widgets.IntRangeSlider(description='Year Range: ', min=1900, max=2020, step=1, value=(1980,1984), style={'description_width': 'initial'}, layout=widgets.Layout(width='400px') )
widget_terms=widgets.Text(description='Filter Terms (comma separated)', value="duurzaam", style={'description_width': 'initial'}, layout=widgets.Layout(width='600px') )
widget_neighbors=widgets.IntSlider(description="Expand Neighborhood Size: ", min=0, max=10, value=5, style={'description_width': 'initial'}, layout=widgets.Layout(width='400px') )

### Display the Widgets to choose the parameters

In [10]:
display(widget_terms)
display(widget_year_range)
display(widget_neighbors)

Text(value='duurzaam', description='Filter Terms (comma separated)', layout=Layout(width='600px'), style=TextS…

IntRangeSlider(value=(1980, 1984), description='Year Range: ', layout=Layout(width='400px'), max=2020, min=190…

IntSlider(value=5, description='Expand Neighborhood Size: ', layout=Layout(width='400px'), max=10, style=Slide…

### Execute the Search

No need to move the code manually here. All parameters are grabbed from the widget values

In [11]:
# Unpack values form Widget
year_from, year_to = widget_year_range.value
years = [str(x) for x in range(year_from, year_to+1)]
FILTER_TERMS = [s.strip() for s in widget_terms.value.split(",")]
# Execute Database Query
where_range = {"year": {"$in": years}}
print(f"Searching terms {FILTER_TERMS} between year {year_from} and {year_to}")
corpus = db.get_corpus(collection, filter_words=FILTER_TERMS, where_obj=where_range, include_embeddings=True, limit=10000)
print(f"Found {len(corpus)} items that match!")

Searching terms ['duurzaam'] between year 1980 and 2020


ValueError: Expected where to have exactly one operator, got {'year_from': 1980, 'year_to': 2020}

### Execute the search for neighborhoods

For each `Passage` in the `Corpus` created with the search result, we will find *k* neighbors and add them to the original `Corpus`.

In [ ]:
import statistics
def get_corpus_with_neighborhoods(collection, corpus, k_neighbors):
    all_neighbors = []
    all_distances = []
    for p in corpus.passages:
        neighbors = db.query_vector_neighbors(collection, vector=p.embedding, k_neighbors=k_neighbors)
        for passage, distance in neighbors:
            all_neighbors.append(passage)
            all_distances.append(distance)
    print(f"Total Datapoints in the neighborhoods = {len(all_neighbors)}")
    print(f"Distance Info: Max = {max(all_distances)} | Min = {min(all_distances)} | Average = {statistics.mean(all_distances)}")
    # Join original passages + new found neighbors
    all_passages = corpus.passages + all_neighbors
    corpus = Corpus(all_passages)
    corpus.embeddings = db.compress_embeddings(corpus)
    print(corpus.embeddings.shape)
    return corpus

corpus = get_corpus_with_neighborhoods(collection, corpus, k_neighbors=widget_neighbors.value)

## Cluster the Corpus

### Choose Minimum Cluster Size

In [ ]:
min_cluster_size_widget = widgets.IntSlider(
    value=10,
    min=5,
    max=len(corpus)//2,
    step=1,
    description='Minimum Cluster Size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'}
)


In [ ]:
min_cluster_size_widget

### Execute Clustering

In [ ]:
# Arguments: min_cluster_size=10, cluster_selection_epsilon=0.1, ...
# See https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html for full list

# e.g. min_samples=10, cluster_selection_epsilon=0.2, cluster_selection_method="leaf"
clusters = corpus.cluster(min_cluster_size=min_cluster_size_widget.value, cluster_selection_epsilon=0.1)
print(f"Found {len(clusters)} clusters in the corpus. (min cluster size is {min_cluster_size_widget.value})")

if len(clusters) > 100:
    raise ValueError("Seems like you have too many clusters! Try with a bigger value for min_cluster_size to avoid memory issues")

for c in clusters:
    print(len(c.passages), c)

### Load Stopwords to avoid including them in the Cluster Labels

In [ ]:
stopwords_file = Path("stopwords-filter-nl.txt")

with open(stopwords_file.absolute(), "rt") as f:
    stopwords = set(f.read().splitlines())

stopwords.update(
    {
        "wij",
        "we",
        "moeten",
        "heer",
        "mevrouw",
        "minister",
        "voorzitter",
        "gaat",
        "wel",
        "den",
    }
)

%autoreload now

### Save Cluster Passages for inspection

In [ ]:
import os
if not os.path.exists("clusters"): 
    os.makedirs("clusters")

selected_metadata = ["year"]

all_clusters_records, df_cluster_labels, df_cluster_meta = [], [], []
for cluster in clusters:
    cluster.set_topic_label(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=5)
    df = cluster.to_dataframe()
    centroid = cluster.centroid()
    label = cluster.label
    cluster_size = len(cluster.passages)
    # Compute Cluster Stats as a Subcorpus
    top_words = " ".join(cluster.top_words(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=10))
    all_clusters_records.append((f"{label}\t{cluster_size}\t{centroid}\t{top_words}\n"))
    df_cluster_labels.append(cluster.label)
    df_cluster_meta.append(df[selected_metadata])
    # Save the Cluster Passages in a File
    file_prefix = f"cluster_{year_from}_{year_to}_{cluster.label.replace('; ', '_')}"
    df.to_csv(f"clusters/{file_prefix}.tsv", sep="\t", index=False) 
    print(cluster)

In [ ]:
with open(f"clusters/clusters_all_{year_from}_{year_to}.csv", "wt") as f:
    f.write("Label\tSize\tCentroid\tTopWords\n")
    for rec in all_clusters_records:
        f.write(rec)

### Plot Cluster Content Distributions

In [ ]:
import matplotlib.pyplot as plt
logging.getLogger('matplotlib').setLevel(logging.WARNING)
from ipywidgets import interact

def plot_cluster_distribution(cluster_index, column_name):

    df = df_cluster_meta[cluster_index]
    cluster_name = df_cluster_labels[cluster_index]

    plt.figure(figsize=(10, 6))
    plt.hist(df[column_name],edgecolor='black')
    plt.xlabel(f'{column_name}', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.title(f'{cluster_name}', fontsize=16)
    plt.show()


cluster_selector = widgets.Dropdown(
    options=[(lbl, i) for i, lbl in enumerate(df_cluster_labels)],
    description='Choose a Cluster:',
    disabled=False,
    style={'description_width': 'initial'} 
)

variable_selector = widgets.Dropdown(
    options=selected_metadata,
    description='Choose a Column to Plot:',
    disabled=False,
    style={'description_width': 'initial'} 
)

interact(plot_cluster_distribution, cluster_index=cluster_selector, column_name=variable_selector)

## Visualize Embeddings (All Clusters)

In [ ]:
import os
from bokeh.io import output_notebook
from bokeh.plotting import show
from tempo_embeddings.visualization.bokeh import BokehInteractiveVisualizer

output_notebook()

meta_fields = corpus.metadata_fields()
meta_fields = ["year", "date", "issue"]

visualizer = BokehInteractiveVisualizer(
    *clusters, metadata_fields=meta_fields, width=2000, height=1000
)

os.environ[
    "BOKEH_ALLOW_WS_ORIGIN"
] = "*"

show(visualizer.create_document)